In [1]:
print("Hello world")

Hello world


In [2]:
%pip install google-adk --quiet
%pip install litellm --quiet
%pip install python-dotenv --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
from dotenv import load_dotenv
load_dotenv() 

True

In [4]:
# Import necessary libraries
import os
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm # For OpenAI support
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types # For creating message Content/Parts
from typing import Optional, Dict, Any

import warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.CRITICAL)

print("Libraries imported.")

Libraries imported.


In [5]:
# Define Model Constants for easier use 
MODEL_GPT = "openai/gpt-4o"

llm = LiteLlm(model=MODEL_GPT)

# Test LLM with a direct call
print(llm.llm_client.completion(model=llm.model, 
                                messages=[{"role": "user", 
                                           "content": "Are you ready?"}], 
                                tools=[]))

print("\nOpenAI is ready for use.")

ModelResponse(id='chatcmpl-ClRp92O40sg0hua6csCd1f5xi6yH7', created=1765424407, model='gpt-4o-2024-08-06', object='chat.completion', system_fingerprint='fp_83554c687e', choices=[Choices(finish_reason='stop', index=0, message=Message(content="Yes, I'm ready. How can I assist you today?", role='assistant', tool_calls=None, function_call=None, provider_specific_fields={'refusal': None}, annotations=[]), provider_specific_fields={})], usage=Usage(completion_tokens=13, prompt_tokens=27, total_tokens=40, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0, text_tokens=None, image_tokens=None), prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=0, cached_tokens=0, text_tokens=None, image_tokens=None)), service_tier='default')

OpenAI is ready for use.


In [ ]:
# ## How to setup Neo4j connection using environment variables
# docker run -d \
#   --name neo4j \
#   -p 7474:7474 -p 7687:7687 \
#   -e NEO4J_AUTH=neo4j/password123 \
#   neo4j:latest

In [ ]:
# if you are coming back after a session restart, start the container again
# docker start neo4j or docker restart neo4j.

In [7]:
## add this to yor .env file
# NEO4J_URI=bolt://localhost:7687
# NEO4J_USERNAME=neo4j
# NEO4J_PASSWORD=password123
# NEO4J_DATABASE=neo4j

In [11]:
%pip install neo4j-for-adk --quiet


ERROR: Could not find a version that satisfies the requirement neo4j-for-adk (from versions: none)
ERROR: No matching distribution found for neo4j-for-adk
Note: you may need to restart the kernel to use updated packages.


In [12]:
# Convenience libraries for working with Neo4j inside of Google ADK
from neo4j_for_adk import graphdb

In [13]:
# Sending a simple query to the database
neo4j_is_ready = graphdb.send_query("RETURN 'Neo4j is Ready!' as message")

print(neo4j_is_ready)

{'status': 'success', 'query_result': [{'message': 'Neo4j is Ready!'}]}


In [14]:
# Define a basic tool -- send a parameterized cypher query
def say_hello(person_name: str) -> dict:
    """Formats a welcome message to a named person. 

    Args:
        person_name (str): the name of the person saying hello

    Returns:
        dict: A dictionary containing the results of the query.
              Includes a 'status' key ('success' or 'error').
              If 'success', includes a 'query_result' key with an array of result rows.
              If 'error', includes an 'error_message' key.
    """
    return graphdb.send_query("RETURN 'Hello to you, ' + $person_name AS reply",
    {
        "person_name": person_name
    })


In [15]:
# Example tool usage (optional test)
print(say_hello("Harsh"))

{'status': 'success', 'query_result': [{'reply': 'Hello to you, Harsh'}]}


**Optional Reading**

- An `Agent` in Google ADK orchestrates the interaction between the user, the LLM, and the available tools

- you configure it with several key parameters:
    * `name`: A unique identifier for this agent (e.g., "friendly_cypher_agent\_v1").  
    * `model`: Specifies which LLM to use. you'll use the `llm` variable we defined above.  
    * `description`: A summary of the agent's overall purpose. This is like public documentation that helps other agents decide when to delegate tasks to *this* agent.  
    * `instruction`: Detailed guidance given to the LLM on how this agent should behave, its persona, goals, and specifically *how and when* to utilize its assigned `tools`.  
    * `tools`: A list containing the actual Python tool functions the agent is allowed to use (e.g., `[say_hello]`).

**Best Practice:** 
- Provide clear and specific `instruction` prompts. The more detailed the instructions, the better the LLM can understand its role and how to use its tools effectively. Be explicit about error handling if needed.
- Choose descriptive `name` and `description` values. These are used internally by ADK and are vital for features like automatic delegation (covered later).